# Hierarchial Indexing 

## 🧩 Understanding MultiIndex in Pandas — The Power of Hierarchical Data

When working with real-world datasets, data often has **multiple levels of grouping or hierarchy**. For example:

- A company’s sales data may be organized by **Region → Store → Month**.
- A school’s performance data may be structured as **Class → Subject → Exam**.

Handling such **multi-level relationships** in a flat table can be cumbersome.

That’s where **Pandas MultiIndex** comes in.

---

## 💡 What is a MultiIndex?

A **MultiIndex** (also known as a **hierarchical index**) allows a DataFrame or Series to have **two or more index levels**.

This makes it easier to represent **multi-dimensional data in 2D tabular form** — without losing the relationships between levels.

In other words, a MultiIndex lets you:

- Group data hierarchically.
- Perform complex selections and aggregations intuitively.
- Work with higher-dimensional data using 2D structures.
---

## 🧱 Creating a MultiIndex DataFrame

Let’s start by manually creating a MultiIndex DataFrame.

In [2]:
import pandas as pd

# Define multiple index levels
arrays = [
    ['North', 'North', 'South', 'South'],
    ['Store A', 'Store B', 'Store A', 'Store B']
]

# Create MultiIndex
index = pd.MultiIndex.from_arrays(arrays, names=("Region", "Store"))

# Create DataFrame
data = pd.DataFrame({
    'Sales': [25000, 30000, 15000, 20000],
    'Profit': [4000, 5200, 2200, 2700]
}, index=index)

data

Sales  Profit
Region Store                 
North  Store A  25000    4000
       Store B  30000    5200
South  Store A  15000    2200
       Store B  20000    2700

### 🏷️ How It Works

Each row index now has two levels:
- Level 0 → Region
- Level 1 → Store

Pandas displays both levels vertically.

You can access them separately or together — just like nested keys.

---

## 🎯 Accessing Data in a MultiIndex

You can use tuple-style indexing to select specific data.

In [3]:
# Select a single row
data.loc[('North', 'Store A')]

Sales     25000
Profit     4000
Name: (North, Store A), dtype: int64

In [ ]:
# Select all stores in a region
data.loc['North']

,Sales,Profit
Store,,
Store A,25000,4000
Store B,30000,5200


### What if I want data for Store A (across all regions)?

In [ ]:
# Use .xs() (Cross Section)
data.xs('Store A', level='Store', axis=0)

,Sales,Profit
Region,,
North,25000,4000
South,15000,2200


#### 🟢 Explanation:
- `xs()` stands for cross section.
- The argument `'Store A'` specifies the value you want.
- The `level='Store'` tells Pandas to look in that particular index level.

---

## 🧩 Understanding stack() and unstack() in Pandas MultiIndex

When working with MultiIndex DataFrames, you often need to reshape your data — that is, move data between rows and columns to make analysis easier.

That’s exactly what `stack()` and `unstack()` do.


| Function        | What It Does                            | Analogy                              |
| --------------- | --------------------------------------- | ------------------------------------ |
| **`stack()`**   | Moves a column level into the row index | "Stack columns on top of each other" |
| **`unstack()`** | Moves a row index level into columns    | "Spread rows out into columns"       |


In [16]:
# unstack() moves the inner index (by default) to columns.
data.unstack()

Sales          Profit        
Store  Store A Store B Store A Store B
Region                                
North    25000   30000    4000    5200
South    15000   20000    2200    2700

In [ ]:
# You can specify which index level to unstack:
data.unstack(level='Region')

Sales        Profit      
Region   North  South  North South
Store                             
Store A  25000  15000   4000  2200
Store B  30000  20000   5200  2700

In [20]:
# stack() - opposite of unstack()
data_unstacked = data.unstack()
data_unstacked.stack(future_stack=True)

Sales  Profit
Region Store                 
North  Store A  25000    4000
       Store B  30000    5200
South  Store A  15000    2200
       Store B  20000    2700

### 🎯 Real-Life Analogy

Think of your dataset as a pivot table in Excel:
- `unstack()` → Makes your table wider, turning row categories into columns.
- `stack()` → Makes your table taller, turning column categories into rows.

| Benefit                          | Description                                                                         |
| -------------------------------- | ----------------------------------------------------------------------------------- |
| **Reshape for analysis**         | Easily convert between wide and long forms for modeling, plotting, or aggregations. |
| **Cleaner data transformations** | Pivot without manually writing loops or merges.                                     |
| **Compatibility**                | Works seamlessly with `groupby`, `pivot_table`, and hierarchical summaries.         |
| **Reversibility**                | `stack()` and `unstack()` are perfect inverses — easy to go back and forth.         |


In [ ]:
# let us now use swaplevel()
data.swaplevel()

,,Sales,Profit
Store,Region,,
Store A,North,25000,4000
Store B,North,30000,5200
Store A,South,15000,2200
Store B,South,20000,2700


| Operation         | Action                | Shape  | Use Case                                     |
| ----------------- | --------------------- | ------ | -------------------------------------------- |
| **`unstack()`**   | Moves index → columns | Wider  | When comparing subcategories side by side    |
| **`stack()`**     | Moves columns → index | Longer | When performing grouped analysis or plotting |
| **`swaplevel()`** | Swaps index order     | Same   | When adjusting index hierarchy               |


----

## 🧮 Working with MultiIndex Columns

MultiIndex isn’t just for rows — it can also be applied to columns.

In [24]:
columns = pd.MultiIndex.from_product(
    [['2024', '2025'], ['Sales', 'Profit']],
    names=['Year', 'Metric']
)

data2 = pd.DataFrame(
    [[25000, 4000, 27000, 4200],
     [15000, 2200, 18000, 2500]],
    index=['Store A', 'Store B'],
    columns=columns
)

print(data2)


Year      2024          2025       
Metric   Sales Profit  Sales Profit
Store A  25000   4000  27000   4200
Store B  15000   2200  18000   2500


In [27]:
# Get Sales data for 2024

data2['2024','Sales']

Store A    25000
Store B    15000
Name: (2024, Sales), dtype: int64

In [ ]:
# Access data using [] 
data2['2024']

Metric,Sales,Profit
Store A,25000,4000
Store B,15000,2200


In [32]:
# Access the inner one usinf xs()
data2.xs('Sales', level='Metric', axis=1)

Year,2024,2025
Store A,25000,27000
Store B,15000,18000


In [ ]:
# let us now stack the data at the level of Metric 
data2_stacked = data2.stack(level='Metric', future_stack=True)
print(data2_stacked)

Year             2024   2025
        Metric              
Store A Sales   25000  27000
        Profit   4000   4200
Store B Sales   15000  18000
        Profit   2200   2500


---

## 🪄 Converting Regular Index to MultiIndex

You can also convert existing columns into a MultiIndex using set_index().

In [33]:
df = pd.DataFrame({
    'Region': ['North', 'North', 'South', 'South'],
    'Store': ['A', 'B', 'A', 'B'],
    'Sales': [25000, 30000, 15000, 20000]
})

multi_df = df.set_index(['Region', 'Store'])
print(multi_df)

              Sales
Region Store       
North  A      25000
       B      30000
South  A      15000
       B      20000


This is particularly useful after grouping or merging data.

In [34]:
# Revert back 
multi_df.reset_index(inplace=True)

In [35]:
multi_df

,Region,Store,Sales
0,North,A,25000
1,North,B,30000
2,South,A,15000
3,South,B,20000


---

## ⚙️ Operations with MultiIndex

Let’s look at some common operations:

In [ ]:
# 1. Sort by index levels
data.sort_index(level='Store')

,,Sales,Profit
Region,Store,,
North,Store A,25000,4000
South,Store A,15000,2200
North,Store B,30000,5200
South,Store B,20000,2700


In [37]:
# 2. Swap levels
data.swaplevel()

,,Sales,Profit
Store,Region,,
Store A,North,25000,4000
Store B,North,30000,5200
Store A,South,15000,2200
Store B,South,20000,2700


In [39]:
# 3. Aggregate across levels
data.groupby(level='Region').sum()

,Sales,Profit
Region,,
North,55000,9200
South,35000,4900


In [40]:
# Aggregation can be done using any custom functions too ! 
import numpy as np

def coeff_var(x):
    return np.std(x) / np.mean(x)

data.groupby(level='Region').agg(coeff_var)

,Sales,Profit
Region,,
North,0.090909,0.130435
South,0.142857,0.102041


---